Data
Amazon fine food reviews from Kaggle

In [39]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Adding Attention Layer as its not a part of the keras
https://www.kaggle.com/kweku20/attention

In [40]:
from shutil import copyfile
copyfile(src = "/kaggle/input/attention/attention.py", dst = "/kaggle/working/attention.py")
from attention import AttentionLayer

Loading the data

In [41]:
class LoadData():
    def __init__(self):
        data = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv")
        print(data.head())
        self.data = data.drop(["Id","ProductId","UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Score","Time"],axis=1)
        

Calling load data object

In [42]:
load_data = LoadData()
data = load_data.data

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [43]:
class PreprocessingData():
    def __init__(self):
        self.max_in_len = 100
        self.max_tar_len = 10
        
    def preprocess_data(self,data):
        data.dropna(axis=0,inplace=True)
        data['Summary'] = data['Summary'].apply(lambda x : 'start '+ x + ' end')
        return data
    
    def get_data(self,data):
        x_train,x_val,y_train,y_val = train_test_split(np.array(data['Text']),np.array(data['Summary']),test_size=0.1,random_state=0,shuffle=True)
        return x_train,x_val,y_train,y_val
    
    def encode_data(self,data,x_train,x_val,y_train,y_val):
        
        # Input Encoding
        in_tokenizer = Tokenizer() 
        in_tokenizer.fit_on_texts(data["Text"].tolist())

        x_train_seq = in_tokenizer.texts_to_sequences(x_train) 
        x_val_seq = in_tokenizer.texts_to_sequences(x_val)

        x_train = pad_sequences(x_train_seq, maxlen = self.max_in_len, padding='post')
        x_val = pad_sequences(x_val_seq, maxlen = self.max_in_len, padding='post')

        self.in_voc = len(in_tokenizer.word_counts) + 1
        
        # Target Encoding
        tar_tokenizer = Tokenizer() 
        tar_tokenizer.fit_on_texts(data["Summary"].tolist())

        y_train_seq = tar_tokenizer.texts_to_sequences(y_train) 
        y_val_seq = tar_tokenizer.texts_to_sequences(y_val)

        y_train = pad_sequences(y_train_seq,  maxlen = self.max_tar_len, padding='post')
        y_val = pad_sequences(y_val_seq, maxlen = self.max_tar_len, padding='post')

        self.tar_voc = len(tar_tokenizer.word_counts) + 1
        return x_train,x_val,y_train,y_val

Calling preprocessing module on loading data

In [44]:
preprocessing_data = PreprocessingData()
data = preprocessing_data.preprocess_data(data)
x_train,x_val,y_train,y_val = preprocessing_data.get_data(data)
x_train,x_val,y_train,y_val = preprocessing_data.encode_data(data,x_train,x_val,y_train,y_val)

TypeError: unsupported operand type(s) for +: 'collections.OrderedDict' and 'int'

Model Creation

In [ ]:
class Model():
    def __init__(self):
        self.model = None
        
    def define_model(self):
        raise NotImplementedError